# API Processing

Below is an example of hitting the Domain (Au Real Estate) API endpoint to validate an address, and see if it is for sale!

1. Hit the [Domain Developer website](https://developer.domain.com.au/).
2. Sign up, and follow the steps to create a project.
3. In 'API Access' on the project page, ensure both 'Agents & Listings' and 'Properties & Locations' are selected.
4. Go to 'Credentials' on project page, and get your client id & secret.

Leveraging [example](https://colab.research.google.com/drive/16wg0VWPim1-dwt7W8jZuJrqFRMrW8iQ0#scrollTo=DqEx2ror5Ih9).

In [1]:
import json
import requests 
import re, string, timeit
import time

In [2]:
from creds import DOMKEY, DOMSECKEY
CLIENT_ID = DOMKEY
CLIENT_SECRET = DOMSECKEY

In [3]:
response = requests.post('https://auth.domain.com.au/v1/connect/token', 
                         data = {'client_id':CLIENT_ID,
                                 "client_secret":CLIENT_SECRET,
                                 "grant_type":"client_credentials",
                                 "scope":"api_listings_read api_addresslocators_read",
                                 "Content-Type":"text/json"},
                        verify = False)
token=response.json()
access_token=token["access_token"]
auth = {"accept":"text/json", "Authorization":"Bearer "+access_token}

/Users/mitchsa/Library/Python/3.7/lib/python/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
# https://colab.research.google.com/drive/16wg0VWPim1-dwt7W8jZuJrqFRMrW8iQ0#scrollTo=tlwYltsjF3DO


##Search Latest Parra listing
##save details (id etc)
##loop through ids to find min/max price (as per example)
##save table

##do daily? limit ids or use 'updatedSince'? to avoid duplicates - or just add and drop dupes?

In [103]:
import pandas as pd
para = pd.DataFrame()

url = "https://api.domain.com.au/v1/listings/residential/_search" 
post_fields ={
      "listingType":"Sale", #Rent

      "propertyType":['House',
                      'NewApartments', 
                      'apartmentUnitFlat'],
    "minPrice":650000,
    "maxPrice":1000000,
    "minBedrooms":2,
    "maxBedrooms":4,
    "minBathrooms":2,
    "maxBathrooms":4,
    "page": 1,
    "pageSize": 100,
    "locations":[
        {
          "state":"NSW",
          "region":"",
          "area":"",
          "suburb":"Parramatta",
          "postCode":"",
          "includeSurroundingSuburbs":False
        }
      ],
    "updatedSince": "2019-12-24",
    "sort": {
        "sortKey":"DateUpdated",
        "direction":"Descending"
    }
}
request = requests.post(url,headers=auth,json=post_fields, verify = False)
# request.content ##401

if not request.ok:
        print(request.content)
#         time.sleep(1)
#         continue
if request.json()!=[]:
    for listing in request.json():
        if listing.get('listing'):
            data = listing.get('listing')
            row = {'id': data.get('id'), 
                   'type': data.get('propertyDetails').get('propertyType'),
                   'price': data.get('priceDetails').get('displayPrice'),
                   'url': 'https://www.domain.com.au/{}-{}-{}'.format(data.get('propertyDetails').get('displayableAddress').\
                                    replace(' ', '-').replace('/','-').replace(',', '-').lower(),
    #                                 data.get('propertyDetails').get('suburb'),
                            data.get('propertyDetails').get('postcode'),
                            data.get('id')),
                 'photo': data.get('media')[0].get('url'), ##add -w4500-h3000 to the end to define the size
                 'lat': data.get('propertyDetails').get('latitude'),
                 'lon': data.get('propertyDetails').get('longitude'),
                 'add': data.get('propertyDetails').get('displayableAddress'),
                 'text': data.get('headline'), 
                 'desc': data.get('summaryDescription'),
                  'bath': data.get('propertyDetails').get('bathrooms'),
                  'bed': data.get('propertyDetails').get('bedrooms'),
                  'car': data.get('propertyDetails').get('carspaces'),
                  }
    #         print(row)
            para = para.append(row, ignore_index = True)
time.sleep(1)





In [104]:
para

,add,bath,bed,car,desc,id,lat,lon,photo,price,text,type,url
0,"302/9 Hassall, Parramatta",2.0,2.0,1.0,<b></b><br />If you're looking for a two bedro...,2.016027e+09,-33.818615,151.008148,https://bucket-api.domain.com.au/v1/bucket/ima...,Contact Agent,Looking for a two bedroom apartment in a top q...,NewApartments,https://www.domain.com.au/302-9-hassall--parra...
1,"2801/88 Church St, Parramatta",2.0,2.0,1.0,<b></b><br />Locates in prestige West village ...,2.016026e+09,-33.819935,151.004000,https://bucket-api.domain.com.au/v1/bucket/ima...,"$805,000",Prime Location! Perfect Layout Two Bedroom for...,ApartmentUnitFlat,https://www.domain.com.au/2801-88-church-st--p...
2,"101/23 Hassall Street, Parramatta",2.0,2.0,1.0,<b></b><br />Set in a high-end executive compl...,2.016026e+09,-33.817830,151.009918,https://bucket-api.domain.com.au/v1/bucket/ima...,JUST LISTED,BRAND NEW 2 BEDROOM APARTMENT WITH HUGE OUTDOO...,ApartmentUnitFlat,https://www.domain.com.au/101-23-hassall-stree...
3,"1011/36 Cowper Street, Parramatta",2.0,2.0,1.0,<b></b><br />PROPERTY LAUNCH 1ST FEBRUARY 2020...,2.016022e+09,-33.820160,151.007034,https://bucket-api.domain.com.au/v1/bucket/ima...,"Price Guide: $660,000 to $685,000",INVEST OR ENJOY THE LIFESTYLE! OPEN TO VIEW SA...,ApartmentUnitFlat,https://www.domain.com.au/1011-36-cowper-stree...
4,"2301/88 Church St, Parramatta",2.0,2.0,1.0,<b></b><br />Stunning residential tower in Par...,2.016019e+09,-33.819935,151.004000,https://bucket-api.domain.com.au/v1/bucket/ima...,"$840,000",BEST PRICE! Brand New 2 Bed Apartment in Parra...,ApartmentUnitFlat,https://www.domain.com.au/2301-88-church-st--p...
5,"1705/88 Church St, Parramatta",2.0,2.0,1.0,<b></b><br />Stunning residential tower in Par...,2.016019e+09,-33.819935,151.004000,https://bucket-api.domain.com.au/v1/bucket/ima...,"$805,000",MUST SELL! Luxury Apartment in Parramatta,ApartmentUnitFlat,https://www.domain.com.au/1705-88-church-st--p...
6,"220 Railway Street, Parramatta",2.0,3.0,1.0,<b></b><br />What a fabulous offering we have ...,2.016017e+09,-33.824030,150.989059,https://bucket-api.domain.com.au/v1/bucket/ima...,AUCTION,PRESENTED TO PERFECTION - OPEN TO VIEW SAT 25/...,House,https://www.domain.com.au/220-railway-street--...
7,"110/127 Pennant Street, Parramatta",2.0,2.0,1.0,<b></b><br />The Collett includes the highest ...,2.015657e+09,-33.807650,151.020828,https://bucket-api.domain.com.au/v1/bucket/ima...,"650,000",Boutique Low-Rise Apartment - Ready To Move In!,ApartmentUnitFlat,https://www.domain.com.au/110-127-pennant-stre...
8,"14/9-11 Cowper Street, Parramatta",2.0,3.0,1.0,<b></b><br />This apartment boasts spacious b...,2.014273e+09,-33.821690,151.006409,https://bucket-api.domain.com.au/v1/bucket/ima...,"$749,000",3 bedrooms Apartment in good location,NewApartments,https://www.domain.com.au/14-9-11-cowper-stree...
9,"3505/88 Church St, Parramatta",2.0,2.0,1.0,<b></b><br />Stunning residential tower in Par...,2.016001e+09,-33.819935,151.004000,https://bucket-api.domain.com.au/v1/bucket/ima...,"$827,000",QUICK SELL! Brand New 2 Bed with Study,ApartmentUnitFlat,https://www.domain.com.au/3505-88-church-st--p...


In [107]:
##this is trying to find the MINIMUM price by shifting max price up until id appears

pid = para.id[0]
starting_max_price=650000
increment=50000
# when starting min price is zero we'll just use the lower bound plus 400k later on
starting_min_price=0

max_price=starting_max_price
searching_for_price=True

ids = para.id

##Note the below for 100 properties will likely hit 500 limit pretty quick.
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale", #Rent

      "propertyType":['House',
                      'NewApartments', 
                      'apartmentUnitFlat'],
    "minPrice":600000,
    "maxPrice":max_price,
    "minBedrooms":2,
    "maxBedrooms":4,
    "minBathrooms":2,
    "maxBathrooms":4,
    "page": 1,
    "pageSize": 100,
    "locations":[
        {
          "state":"NSW",
          "region":"",
          "area":"",
          "suburb":"Parramatta",
          "postCode":"",
          "includeSurroundingSuburbs":False
        }
      ],
    "updatedSince": "2019-12-24",
    "sort": {
        "sortKey":"DateUpdated",
        "direction":"Descending"
        }
    }

    request = requests.post(url,headers=auth,json=post_fields, verify = False)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings
    
#     for pid in ids
    if int(pid) in listings:
            max_price=max_price-increment
            print("Lower bound found: ", max_price)
            searching_for_price=False
    else:
        max_price=max_price+increment
        print("Not found. Increasing max price to ",max_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly   

Not found. Increasing max price to  700000
Lower bound found:  650000


In [ ]:
searching_for_price=True
if starting_min_price>0:
    min_price=starting_min_price
else:  
    min_price=max_price+400000  

In [ ]:
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "minPrice":min_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            min_price=min_price+increment
            print("Upper bound found: ", min_price)
            searching_for_price=False
    else:
        min_price=min_price-increment
        print("Not found. Decreasing min price to ",min_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly     
       

In [105]:
prop_id = "2015989681"
url = "https://api.domain.com.au/v1/listings/"+prop_id
auth = {"Authorization":"Bearer "+access_token}
request = requests.get(url,headers=auth, verify = False)
request.content
# r=request.json()

b'{"objective":"sale","propertyTypes":["apartmentUnitFlat"],"status":"live","saleMode":"buy","channel":"residential","addressParts":{"stateAbbreviation":"nsw","displayType":"fullAddress","streetNumber":"88","unitNumber":"605","street":"Church St","suburb":"Parramatta","postcode":"2150","displayAddress":"605/88 Church St, Parramatta NSW 2150"},"advertiserIdentifiers":{"advertiserType":"agency","advertiserId":28976,"contactIds":[1717706],"agentIds":["A120836"]},"bathrooms":2.0,"bedrooms":2.0,"carspaces":1.0,"dateUpdated":"2020-01-23T04:37:05.397Z","dateListed":"2020-01-06T05:24:12Z","description":"Stunning residential tower in Parramatta\'s CBD, with rooftop entertaining deck with beautiful landscaping. \\r\\n\\r\\nPodium level rooftop garden with pocket parks, shelters, seating, grassy terraces and BBQ areas. Ground floor retail and intimate alfresco dining precinct. New multi-billion dollar transport networks including West Connex, the new Metro West fast train and the new airport furt

In [82]:
# r

<hr>

In [8]:
import warnings 
warnings.filterwarnings('ignore')
import time
import pandas as pd
map_base = pd.DataFrame()

for i in range(1,3):
    for j in range(1,3):
        if (i%2==0)&(j%20==0):
            print(i, j)
        url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
        post_fields ={
              "listingType":"Sale", #Rent
              "page":i,
              "propertyType":['House',
                              'NewApartments', 
                              'apartmentUnitFlat'],
            "minPrice":650000,
            "maxPrice":1000000,
            "minBedrooms":2,
            "maxBedrooms":4,
            "minBathrooms":2,
            "maxBathrooms":4,
            "pageSize":j,#100,
            "locations":[
                {
                  "state":"NSW",
                  "region":"",
                  "area":"",
                  "suburb":"Parramatta",
                  "postCode":"",
                  "includeSurroundingSuburbs":False
                }
              ]
            }
        request = requests.post(url,headers=auth,json=post_fields, verify = False)
        if not request.ok:
            print(request.content)
            time.sleep(1)
            continue
        if request.json()!=[]:
            if request.json()[0].get('listing'):
                data = request.json()[0].get('listing')
                row = {'id': data.get('id'), 
                       'type': data.get('propertyDetails').get('propertyType'),
                       'price': data.get('priceDetails').get('displayPrice'),
                       'url': 'https://www.domain.com.au/{}-{}-{}'.format(data.get('propertyDetails').get('displayableAddress').\
                                        replace(' ', '-').replace('/','-').replace(',', '-').lower(),
#                                 data.get('propertyDetails').get('suburb'),
                                data.get('propertyDetails').get('postcode'),
                                data.get('id')),
                     'photo': data.get('media')[0].get('url'), ##add -w4500-h3000 to the end to define the size
                     'lat': data.get('propertyDetails').get('latitude'),
                     'lon': data.get('propertyDetails').get('longitude'),
                     'add': data.get('propertyDetails').get('displayableAddress'),
                     'text': data.get('headline'), 
                     'desc': data.get('summaryDescription'),
                      'bath': data.get('propertyDetails').get('bathrooms'),
                      'bed': data.get('propertyDetails').get('bedrooms'),
                      'car': data.get('propertyDetails').get('carspaces'),
                      }
                map_base = map_base.append(row, ignore_index = True)
        time.sleep(1)

In [12]:
map_base

,add,bath,bed,car,desc,id,lat,lon,photo,price,text,type,url
0,"101/23 Hassall Street, Parramatta",2.0,2.0,1.0,<b></b><br />Set in a high-end executive compl...,2.016026e+09,-33.81783,151.009918,https://bucket-api.domain.com.au/v1/bucket/ima...,JUST LISTED,BRAND NEW 2 BEDROOM APARTMENT WITH HUGE OUTDOO...,ApartmentUnitFlat,https://www.domain.com.au/101-23-hassall-stree...
1,"1011/36 Cowper Street, Parramatta",2.0,2.0,1.0,<b></b><br />PROPERTY LAUNCH 1ST FEBRUARY 2020...,2.016022e+09,-33.82016,151.007034,https://bucket-api.domain.com.au/v1/bucket/ima...,"Price Guide: $660,000 to $685,000",INVEST OR ENJOY THE LIFESTYLE! OPEN TO VIEW SA...,ApartmentUnitFlat,https://www.domain.com.au/1011-36-cowper-stree...


In [9]:
tbll = map_base.drop_duplicates()

In [10]:
len(map_base.drop_duplicates())

2

In [188]:
tbll['price2'] = [int(''.join([s for s in g if s.isdigit()][:3])) if 
                  [s for s in g if s.isdigit()][:3] and int(''.join([s for s in g if s.isdigit()][:3]))>500 else 'Contact Agent' 
                  for g in tbll.price]
import numpy as np
tbll['price3'] = [int(x)  if x!='Contact Agent' else 0 for x in tbll['price2']]
tbll['price4'] = np.where(tbll['price3']==0, 'Contact Agent',
                             np.where(tbll['price3']<=650, '650k', 
                                  np.where(tbll['price3']<=700, '700k', 
                                       np.where(tbll['price3']<=750, '750k', 
                                           np.where(tbll['price3']<=800, '800k',
                                               np.where(tbll['price3']<=900, '900k', 0))))))


In [212]:
tbll['url2'] = [f'<a href = "{x}" target = "_blank">Original Source</a>' for x in tbll['url']]

In [189]:
tbll.price4.value_counts()

Contact Agent    22
700k             12
650k              7
750k              6
800k              4
900k              3
Name: price4, dtype: int64

In [6]:
# import os
# os.getcwd()

In [11]:
import pandas as pd
tbll = pd.read_csv('API/ParraRE.csv')
tbll['url2'] = [f'<a href = "{x}" target = "_blank">Original Source</a>' for x in tbll['url']]

In [30]:
# tbl = pd.concat([map_df,map_base]).drop_duplicates()

In [9]:
# map_df = map_base.drop_duplicates()

In [22]:
def popup_func(row, string_var = []):
    import folium
    from folium import IFrame
    import base64
    from io import BytesIO
    from PIL import Image
    import numpy as np
    
    """Add row column names to include that information in popup bubbles."""
    st = ""
    for item in string_var:
        st = st+str(item.capitalize())+": "+str(row[item])+"<br>"
    
    from PIL import Image
    # import requests
    from io import BytesIO

    import warnings 
    warnings.filterwarnings('ignore')
    response = requests.get(row['photo'], verify = False)
    img = Image.open(BytesIO(response.content))
#     img    
#     image_object = work_bucket.Object(row.img)
#     in_mem =    io.BytesIO(image_object.get()['Body'].read())
#     img = IM.open(in_mem)
    h,w = img.size
        
    img.thumbnail((np.round(h/5), np.round(350)))
        #1, 350) #np.round((350/w)*h)
        #(np.round(h/5), np.round(w/5)))
    imgByteArr = BytesIO()
    img.save(imgByteArr, format='PNG', optimize=True,quality=10)
    
    
    img_str = base64.b64encode(imgByteArr.getvalue()).decode()
    #base64.b64encode(open("{}".format(row.file), 'rb').read())
    pop = folium.Popup(IFrame(
                '<div style="font-family: Arial">{}<br><img src="data:image/;base64,{}">'.format(str(st),
                                                                                                
                                                                                          img_str)+
                '</div>',
                                              width = 350, height = 350

                                              ), 
                                                       max_width=2650)
    return(pop)

In [23]:
def icon_func(map_df, group_col, colour_shift = 0):
    """
    Returns icons, icon colours, and an appropriate legend.
    """
    if(colour_shift>18):
        print("To big of colour spectrum shift")
        return
    
    fa = ['random', 'cog', 'magnet', 'pencil', 'wrench', 'signal',  
          'tag', 'certificate', 'volume-off',  'th-large', 
          'map-marker', 'plus', 
         'file', 'refresh', 'align-center', 'link', 
     'repeat', 'th', 'heart', 'briefcase']
    
    cols = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
                 'darkpurple', 'pink', 'lightblue', 'lightgreen',
                 'gray', 'black', 'lightgray']*2
    
    ics = ['Contact Agent', '600k', '650k', '700k', '750k', '800k', '900k']#map_df[group_col].unique()
    
    if(len(ics)>len(fa)):
        print("Too many groups")
        return
    
    
    icns = dict(zip(ics,
                fa[:len(ics)]))
    clrs = dict(zip(ics, 
                cols[colour_shift:(colour_shift+len(ics))]))
    
    leg = ""
    for key, item in icns.items():
        leg = leg + """{}&nbsp; <i class="fa fa-{} fa-2x"
                       style="color:{}"></i><br><br>""".format(key, item, clrs[key])
    legh = len(icns.items())*56
    
    legend_html = """
         <div style="position: fixed; 
         bottom: 25px; left: 25px; width: 120px; height: {}px; text-align:center;
         border:2px solid grey; background-color: white; z-index:9999; font-size:14px;
         "><h4 style = "text-align:center">Legend</h4>
         {}
          </div>
         """.format(legh, leg[:-8])
    
    return(icns, clrs, legend_html)

In [24]:
def map(map_df, lat_col, lon_col, group_col, strings = [], colour_shift = 0):
    import folium
    focus = map_df.groupby(lambda _ : True).agg({lat_col:'mean', lon_col:'mean'}).iloc[0]
    lat,lon = focus[lat_col], focus[lon_col] 
    f = folium.Figure(height=600)
    m = folium.Map(location=[lat, lon], zoom_start = 16)#,  max_zoom=max_zoom)
    
    icns, clrs, legend_html = icon_func(map_df=map_df, 
                                        group_col = group_col, 
                                        colour_shift = colour_shift)
    
    map_df.apply(lambda row:folium.Marker(location=[row[lat_col], row[lon_col]],
                                            popup = popup_func(row, strings),
                                            icon = folium.Icon(color=clrs[row[group_col]], 
                                                               icon_color = "white",
                                                               prefix = 'fa',
                                                               icon = icns[row[group_col]])
                                               ).add_to(m), axis=1) 
    h = folium.Marker(location=[-33.81168,151.0113152],
                      icon = folium.Icon(color = 'red'))
    m.add_child(h)
#     m.add_child(marker_cluster)
    f.add_child(m)
    f.get_root().html.add_child(folium.Element(legend_html))
    f.save("{}.html".format('test'))
    return f

In [25]:
map(tbll[:10], 'lat', 'lon', 'price4', 
    ['add', 'bed', 'bath', 'car', 'text', 'desc', 'url2'])

Api's and maps done!
<hr>

In [ ]:
##Build Map    
def map(map_df, 
            lat_col = 'LATITUDE', lon_col = 'LONGITUDE', 
            group_col = 'PLEASE SELECT', #e.g. 'PRIMARY_TECHNOLOGY'
            string_var = [], #e.g. ['LOCATION_ID', 'PRIMARY_TECHNOLOGY']
            
            height = 900,
            zoom = 10,
            colour_shift = 0,
            disable_cluster = None,
            max_zoom = 18,
            
            save_map_name = "",
            disable_legend = False,
            geocode = False,
            
            icon_func = icons, 
            popup_func = popup
            ):
    
    import folium
#     from folium.plugins import MarkerCluster
    
    
    height = height
    zoom = zoom
    if disable_cluster is None:
        disable_cluster = zoom+2
    
    
    focus = map_df.groupby(lambda _ : True).agg({lat_col:'mean', lon_col:'mean'}).iloc[0]
    lat,lon = focus[lat_col], focus[lon_col] 
    
    icns, clrs, legend_html = icon_func(map_df=map_df, 
                                        group_col = group_col, 
                                        colour_shift = colour_shift)
    
   
    f = folium.Figure(height=height)
    m = folium.Map(location=[lat, lon], zoom_start = zoom,  max_zoom=max_zoom)
    marker_cluster = MarkerCluster(options = {'disableClusteringAtZoom':disable_cluster}) 

    
    if(geocode==True):
    
        from geopy.geocoders import Nominatim
        from geopy.exc import GeocoderTimedOut
        geolocator = Nominatim(user_agent="hmm")
    
        def do_geocode(lat, lon):
            try:
                return geolocator.reverse("{}, {}".format(lat, lon), timeout=600)
            except GeocoderTimedOut:
                return geolocator.reverse("{}, {}".format(lat, lon), timeout=600)
    
        if(len(map_df)>500):
            print('- Geocoding may take some time as there are over 500 addresses -\n')
        print("- Reverse Geocoding -")
        map_df.loc[:,('GEOCODE')] = map_df.apply(lambda row:("<br>"+do_geocode(row[lat_col], row[lon_col]).address), axis = 1)
        string_var = string_var+['GEOCODE']
        
    
    
    map_df.apply(lambda row:folium.Marker(location=[row[lat_col], row[lon_col]],
                                            popup = popup_func(row, string_var),
                                            icon = folium.Icon(color=clrs[row[group_col]], 
                                                               icon_color = "white",
                                                               prefix = 'fa',
                                                               icon = icns[row[group_col]])
                                               ).add_to(marker_cluster), axis=1) 


    m.add_child(marker_cluster)
    f.add_child(m)
    
    if(disable_legend == False):
        f.get_root().html.add_child(folium.Element(legend_html))
    
    if(save_map_name != ""):
        f.save("{}.html".format(save_map_name))
    
    return(f)

In [ ]:
##To provide some quick mapping options.

##Define what goes in the popup bubble
def popup(row, var = []):
    import folium
    
    """Add row column names to include that information in popup bubbles."""
    st = ""
    for item in var:
        st = st+str(item)+": "+str(row[item])+"<br>"
    
    pop = folium.Popup(folium.IFrame(
                    '<div style="font-family: Arial">{}</div>'.format(str(st)),

                                                    width = 300, height = 120))
    return(pop)
    
    
##Define popup colours and icons
def icons(map_df, group_col, colour_shift = 0):
    """
    Returns icons, icon colours, and an appropriate legend.
    """
    if(colour_shift>18):
        print("To big of colour spectrum shift")
        return
    
    fa = ['random', 'cog', 'magnet', 'pencil', 'wrench', 'signal',  
          'tag', 'certificate', 'volume-off',  'th-large', 
          'map-marker', 'plus', 
         'file', 'refresh', 'align-center', 'link', 
     'repeat', 'th', 'heart', 'briefcase']
    
    cols = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
                 'darkpurple', 'pink', 'lightblue', 'lightgreen',
                 'gray', 'black', 'lightgray']*2
    
    ics = map_df[group_col].unique()
    
    if(len(ics)>len(fa)):
        print("Too many groups")
        return
    
    
    icns = dict(zip(ics,
                fa[:len(ics)]))
    clrs = dict(zip(ics, 
                cols[colour_shift:(colour_shift+len(ics))]))
    
    leg = ""
    for key, item in icns.items():
        leg = leg + """{}&nbsp; <i class="fa fa-{} fa-2x"
                       style="color:{}"></i><br><br>""".format(key, item, clrs[key])
    legh = len(icns.items())*56
    
    legend_html = """
         <div style="position: fixed; 
         bottom: 25px; left: 25px; width: 120px; height: {}px; text-align:center;
         border:2px solid grey; background-color: white; z-index:9999; font-size:14px;
         "><h4 style = "text-align:center">Legend</h4>
         {}
          </div>
         """.format(legh, leg[:-8])
    
    return(icns, clrs, legend_html)
    
    
##Build Map    
def map(map_df, 
            lat_col = 'LATITUDE', lon_col = 'LONGITUDE', 
            group_col = 'PLEASE SELECT', #e.g. 'PRIMARY_TECHNOLOGY'
            string_var = [], #e.g. ['LOCATION_ID', 'PRIMARY_TECHNOLOGY']
            
            height = 900,
            zoom = 10,
            colour_shift = 0,
            disable_cluster = None,
            max_zoom = 18,
            
            save_map_name = "",
            disable_legend = False,
            geocode = False,
            
            icon_func = icons, 
            popup_func = popup
            ):
    """
    An attempt to speed up mapping in folium. May need further abstraction, or more 
    function exposure. Simply have a pandas dataframe with lat/lons, and a grouping 
    column (for icon colouring etc) to produce a useful map. Can also choose row 
    variables to put in the icons' popups, save the map, and disable clustering.
    
    Allows you to use your own custom popup functions.
    """
    
    import folium
    from folium.plugins import MarkerCluster
    
    
    height = height
    zoom = zoom
    if disable_cluster is None:
        disable_cluster = zoom+2
    
    
    focus = map_df.groupby(lambda _ : True).agg({lat_col:'mean', lon_col:'mean'}).iloc[0]
    lat,lon = focus[lat_col], focus[lon_col] 
    
    icns, clrs, legend_html = icon_func(map_df=map_df, 
                                        group_col = group_col, 
                                        colour_shift = colour_shift)
    
   
    f = folium.Figure(height=height)
    m = folium.Map(location=[lat, lon], zoom_start = zoom,  max_zoom=max_zoom)
    marker_cluster = MarkerCluster(options = {'disableClusteringAtZoom':disable_cluster}) 

    
    if(geocode==True):
    
        from geopy.geocoders import Nominatim
        from geopy.exc import GeocoderTimedOut
        geolocator = Nominatim(user_agent="hmm")
    
        def do_geocode(lat, lon):
            try:
                return geolocator.reverse("{}, {}".format(lat, lon), timeout=600)
            except GeocoderTimedOut:
                return geolocator.reverse("{}, {}".format(lat, lon), timeout=600)
    
        if(len(map_df)>500):
            print('- Geocoding may take some time as there are over 500 addresses -\n')
        print("- Reverse Geocoding -")
        map_df.loc[:,('GEOCODE')] = map_df.apply(lambda row:("<br>"+do_geocode(row[lat_col], row[lon_col]).address), axis = 1)
        string_var = string_var+['GEOCODE']
        
    
    
    map_df.apply(lambda row:folium.Marker(location=[row[lat_col], row[lon_col]],
                                            popup = popup_func(row, string_var),
                                            icon = folium.Icon(color=clrs[row[group_col]], 
                                                               icon_color = "white",
                                                               prefix = 'fa',
                                                               icon = icns[row[group_col]])
                                               ).add_to(marker_cluster), axis=1) 


    m.add_child(marker_cluster)
    f.add_child(m)
    
    if(disable_legend == False):
        f.get_root().html.add_child(folium.Element(legend_html))
    
    if(save_map_name != ""):
        f.save("{}.html".format(save_map_name))
    
    return(f)
    
    
    
    
    
    

In [ ]:
def path_to_image(row, string_var = []):
    import folium
    from folium import IFrame
    import base64
    from io import BytesIO
    from PIL import Image as IM
    import numpy as np
    
    """Add row column names to include that information in popup bubbles."""
    st = ""
    for item in string_var:
        st = st+str(item.capitalize())+": "+str(row[item])+"<br>"
    
        
    image_object = work_bucket.Object(row.img)
    in_mem =    io.BytesIO(image_object.get()['Body'].read())
    img = IM.open(in_mem)
    h,w = img.size
        
    img.thumbnail((np.round(h/10), np.round(w/10)))
    imgByteArr = io.BytesIO()
    img.save(imgByteArr, format='PNG', optimize=True,quality=10)
    
    
    img_str = base64.b64encode(imgByteArr.getvalue()).decode()
    #base64.b64encode(open("{}".format(row.file), 'rb').read())
    pop = folium.Popup(IFrame(
                '<div style="font-family: Arial">{}<br><img src="data:image/{};base64,{}">'.format(str(st),
                                                                                               row.img, 
                                                                                          img_str)+
                '</div>',
                                              width = 350, height = 350

                                              ), 
                                                       max_width=2650)
    return(pop)

In [21]:
# # test = pd.DataFrame()
# row = {'id': l.get('id'), 
#                      'photo': l.get('media')[0].get('url'), 
#                      'lat': l.get('propertyDetails').get('latitude'),
#                      'lon': l.get('propertyDetails').get('longitude'),
#                      'add': l.get('propertyDetails').get('displayableAddress'),
#                      'text': l.get('headline'), 
#                      'desc': l.get('summaryDescription')}
# test = test.append(row, ignore_index = True)

In [49]:
test

,add,desc,id,lat,lon,photo,text
0,"1-8/6 O'Reilly Street, Parramatta","<b></b><br />New to the market, this exciting ...",2.015888e+09,-33.819195,150.994949,https://bucket-api.domain.com.au/v1/bucket/ima...,Exciting investment opportunity close to the CBD
1,"23 Pemberton Street, Parramatta",<b></b><br />This property is ready to be reim...,2.015893e+09,-33.810722,151.020187,https://bucket-api.domain.com.au/v1/bucket/ima...,Exciting development opportunity close to the CBD


In [ ]:
{'id': full.get('id'), 
 'photo': full.get('media')[0].get('url'), 
 'lat': full.get('propertyDetails').get('latitude'),
 'lon': full.get('propertyDetails').get('longitude'),
 'add': full.get('propertyDetails').get('displayableAddress'),
 'text': full.get('headline'), 
 'desc': full.get('summaryDescription')}

In [ ]:
if request.json()[0].get('listing'):
    l = request.json()[0].get('listing')
    listings.append(l["id"])
elif request.json()[0].get('listings'):
    l = request.json()[0].get('listings')
    for listing in l:
        listings.append(listing["id"])

In [ ]:
if request.json()[0].get('listing'):
    l = request.json()[0].get('listing')
    listings.append(l["id"])
elif request.json()[0].get('listings'):
    l = request.json()[0].get('listings')
    for listing in l:
        listings.append(listing["id"])
        
for ls in l if ls.get('type')=='PropertyListing'][0].get('listing')

In [ ]:
row_list = []
for i in range (0,5):
    row_list.append(dict( (a,np.random.randint(100)) for a in ['A','B','C','D','E']))
for i in range( 1,numOfRows-4):
    dict1 = dict( (a,np.random.randint(100)) for a in ['A','B','C','D','E'])
    row_list.append(dict1)

df4 = pd.DataFrame(row_list, columns=['A','B','C','D','E'])

In [ ]:
url = l.get('media')[0].get('url')

In [ ]:
from PIL import Image
# import requests
from io import BytesIO

response = requests.get(url, verify = False)
img = Image.open(BytesIO(response.content))
img

In [ ]:
'priceDetails': {'displayPrice': '$630,000-$660,000'},
 'media': [{'category': 'Image',
   'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2015519470_1_1_190813_010911-w1600-h1068'},
  {'category': 'Image',
   'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2015519470_2_1_190813_010911-w1600-h1068'},
  {'category': 'Image',
   'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2015519470_3_1_190813_010911-w1600-h1068'},
  {'category': 'Image',
   'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2015519470_4_1_190813_010911-w1600-h1068'},
  {'category': 'Image',
   'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2015519470_5_1_190813_010911-w1600-h1068'}],

In [ ]:
len(set(listings))

In [ ]:
max_price=starting_max_price
searching_for_price=True

In [ ]:
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "maxPrice":max_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields, verify = False)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if (int(property_id) in listings)|(max_price>1500000):
            max_price=max_price-increment
            print("Lower bound found: ", max_price)
            searching_for_price=False
    else:
        max_price=max_price+increment
        print("Not found. Increasing max price to ",max_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly   

In [ ]:
searching_for_price=True
if starting_min_price>0:
    min_price=starting_min_price
else:  
    min_price=max_price+400000  

In [ ]:
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "minPrice":min_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields, verify = False)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            min_price=min_price+increment
            print("Upper bound found: ", min_price)
            searching_for_price=False
    else:
        min_price=min_price-increment
        print("Not found. Decreasing min price to ",min_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly

In [ ]:
if max_price<1000000:
    lower=min_price/1000
    upper=max_price/1000
    denom="k"
else: 
    lower=min_price/1000000
    upper=max_price/1000000
    denom="m"

In [ ]:
print(da['displayAddress'])
print(r['headline'])
print("Property Type:",property_type_str)
print("Details: ",int(bedrooms),"bedroom,",int(bathrooms),"bathroom,",int(carspaces),"carspace")
print("Display price:",r['priceDetails']['displayPrice'])      
if max_price==min_price:
  print("Price guide:","$",lower,denom)
else:
  print("Price range:","$",lower,"-","$",upper,denom)
print("URL:",r['seoUrl'])

In [ ]:
# url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
# post_fields ={
#       "listingType":"Sale", #Rent
#       "page":1,
#       "propertyType":['apartmentUnitFlat'],
#       "pageSize":4,
#       "locations":[
#         {
#           "state":"NSW",
#           "region":"",
#           "area":"",
#           "suburb":"Parramatta",
#           "postCode":"",
#           "includeSurroundingSuburbs":False
#         }
#       ]
#     }

In [ ]:
#get details
da=r['addressParts']
postcode=da['postcode']
suburb=da['suburb']
bathrooms=r['bathrooms']
bedrooms=r['bedrooms']
carspaces=r['carspaces']
property_type=r['propertyTypes']
print(property_type,postcode, suburb, bedrooms, bathrooms,  carspaces)

# the below puts all relevant property types into a single string. eg. a property listing can be a 'house' and a 'townhouse'
n=0
property_type_str=""
for p in r['propertyTypes']:
    property_type_str=property_type_str+(r['propertyTypes'][int(n)])
    n=n+1
print(property_type_str)  

In [ ]:
# url = "https://api.domain.com.au/v1/addressLocators?searchLevel=Address&streetNumber=100&streetName=Harris&streetType=Street&suburb=Pyrmont&state=NSW&postcode=2009"
url = "https://api.domain.com.au/v1/addressLocators?searchLevel=Address&unitNumber=3&streetNumber=61&streetName=High&streetType=Street&suburb=Parramatta&state=NSW&postcode=2150"

# url = "https://api.domain.com.au/v1/addressLocators?searchLevel=Address&unitNumber=6&streetNumber=65&streetName=Albert&streetType=Crescent&suburb=Burwood&state=NSW&postcode=2134"
# url = "https://api.domain.com.au/v1/addressLocators?searchLevel=Address&unitNumber=1&streetNumber=27&streetName=Stewart&streetType=Street&suburb=PARRAMATTA&state=NSW&postcode=2150"

# url = """https://api.domain.com.au/v1/addressLocators?searchLevel=Address&suburb=Pyrmont&state=NSW&postcode=2009"""
request = requests.get(url,
                       headers=auth, 
                       verify = False)
request.json()

In [ ]:
property_id="2015010473"
starting_max_price=1000000
increment=50000
# when starting min price is zero we'll just use the lower bound plus 400k later on
starting_min_price=0
url = "https://api.domain.com.au/v1/listings/"+property_id
auth = {"accept":"text/json", "Authorization":"Bearer "+access_token}
request = requests.get(url,headers=auth, verify = False)
r=request.json()

In [ ]:
prop_id = request.json()[0].get('ids')[0].get('id')
prop_id

In [ ]:
url = "https://api.domain.com.au/v1/listings/"+str(prop_id)
auth = {"accept":"text/json", "Authorization":"Bearer "+access_token}
request = requests.get(url,headers=auth, verify = False)
# request
r=request.json()
r

In [ ]:
url